In [47]:
from sage.graphs.trees import TreeIterator
import pandas as pd
from itertools import combinations
%run strong_odd_chromatic_number_ILP.ipynb

def generate_unicyclic_graphs(n, max_graphs=None):
    """
    Generira enociklične grafe z n vozlišči, izračuna velikost cikla in močno liho kromatično število.
    
    Parametri:
    - n: Število vozlišč v grafu.
    - max_graphs: Največje število grafov za generacijo (privzeto None pomeni vse možne grafe).
    
    Vrne:
    - tabelo z informacijami o grafih, velikosti cikla in kromatičnem številu.
    """
    results = []
    graphs_generated = 0

    # Uporabimo TreeIterator za generacijo dreves
    trees = TreeIterator(n)

    for tree in trees:
        tree = Graph(tree)  # Pretvorimo drevo v SageMath graf

        # Dodajamo eno povezavo med katerima koli dvema vozliščema, ki še nista povezana
        for u, v in combinations(range(n), 2):
            if not tree.has_edge(u, v):
                graph = tree.copy()
                graph.add_edge(u, v)

                # Preverimo, ali je graf povezan in ima točno en cikel
                if graph.is_connected() and graph.num_edges() == n:
                    cycles = graph.cycle_basis()  # Najdemo vse cikle
                    if len(cycles) == 1:  # Mora biti točno en cikel
                        cycle_size = len(cycles[0])

                        # Izračun močno lihega kromatičnega števila
                        chromatic_number = strong_odd_chromatic_number(graph, max_colors=5)

                        # Dodamo rezultate v tabelo
                        results.append({
                            "Total Vertices": n,
                            "Cycle Size": cycle_size,
                            "Chromatic Number": chromatic_number
                        })

                        graphs_generated += 1
                        if max_graphs is not None and graphs_generated >= max_graphs:
                            return pd.DataFrame(results)

    return pd.DataFrame(results)


In [48]:
def test_unicyclic_graphs(max_vertices, max_graphs=None):
    """
    Testira enociklične grafe za različne velikosti vozlišč in beleži velikost cikla.
    
    Parametri:
    - max_vertices: Največje število vozlišč v grafu.
    - max_graphs: Največje število grafov za posamezno število vozlišč.
    
    Vrne:
    - tabelo z združenimi rezultati.
    """
    all_results = []

    for n in range(3, max_vertices + 1):  # Enociklični grafi obstajajo šele od n=3
        results = generate_unicyclic_graphs(n, max_graphs)
        all_results.append(results)

    # Združimo vse rezultate
    return pd.concat(all_results, ignore_index=True)


In [49]:
# Testiranje
max_vertices = 10 
max_graphs = 10  

# Generacija in testiranje grafov
results = test_unicyclic_graphs(max_vertices, max_graphs)

# Prikaz tabelarnih rezultatov
print(results)

# Povzetek po velikostih ciklov, kromatičnem številu in številu vozlišč
summary = results.groupby(["Total Vertices", "Cycle Size", "Chromatic Number"]).size().reset_index(name="Number of Graphs")

from IPython.display import display  # Za lepši prikaz

display(summary.style.hide(axis="index"))

    Total Vertices  Cycle Size  Chromatic Number
0                3           3                 3
1                4           3                 4
2                4           3                 4
3                4           4                 4
4                4           3                 4
..             ...         ...               ...
62              10           4                 4
63              10           5                 4
64              10           3                 4
65              10           4                 4
66              10           5                 4

[67 rows x 3 columns]


Total Vertices,Cycle Size,Chromatic Number,Number of Graphs
3,3,3,1
4,3,4,5
4,4,4,1
5,3,3,1
5,3,4,5
5,4,4,3
5,5,5,1
6,3,4,4
6,4,4,3
6,5,4,2
